1. Install the Vertex AI SDK: Open a terminal window and enter the command below. You can also [install it in a virtualenv](https://googleapis.dev/python/aiplatform/latest/index.html)

In [1]:
!pip install --upgrade google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 18.4 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.57.0
    Uninstalling google-cloud-aiplatform-1.57.0:
      Successfully uninstalled google-cloud-aiplatform-1.57.0


2. Use the following code in your application to request a model response

In [2]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

def generate():
  vertexai.init(project="corebigenai", location="us-central1")
  model = GenerativeModel(
    "gemini-1.0-pro-001",
  )
  responses = model.generate_content(
      [text1],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
  )

  for response in responses:
    print(response.text, end="")

text1 = """Consider a table called `corebigenai.demos.productos`
with schema as described below.
Column name   Description   type
prod_id      Codigo del Producto (Numero de Producto que identifica el producto)
prod_name      Nombre del producto (Todo en letra minuscula)
prod_name_long      Descripcion del producto (Todo en letra minuscula)
prod_brand     Marca del producto (Todo en letra minuscula)
subcategory    Categoria del producto (Todo en letra minuscula)
tags       Etqiquetas del producto (Todo en letra minuscula)
prod_unit_price      Precio unitario del producto
prod_units   Tipo de unidades del producto (Todo en letra minuscula)
prod_source  Proveedor o Fabricante (Todo en letra minuscula)

Use always lowercase for the words in the description of the product.
Always separe each word with \"OR\" SQL Sentence about product description in the query for have best results.
prod_id is alwasy a number.
As data analysis expert, please write a SQL query in bigquery and use always the SQL comands LIKE with OR for each main word, and always LIMIT 10
about the next request: Cual es el producto mas costoso?"""

generation_config = {
    "max_output_tokens": 2048,
    "temperature": 0.2,
    "top_p": 1,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

generate()



```sql
SELECT
  prod_name,
  prod_unit_price
FROM `corebigenai.demos.productos`
WHERE
  prod_name_long LIKE '%mas%OR%costoso%'
ORDER BY
  prod_unit_price DESC
LIMIT 10;
```

In [ ]:
! pip install pandas==2.0.0

In [1]:
import re
import datetime
import requests
from io import StringIO, BytesIO
from html.parser import HTMLParser
from bs4 import BeautifulSoup
from google import auth
from google.cloud import storage
from google.auth.transport import requests as req
import pandas as pd
from google.cloud import bigquery

In [6]:
client2 = bigquery.Client()

In [12]:
query=" SELECT   prod_name,   prod_unit_price FROM   `corebigenai.demos.productos` WHERE   LOWER(prod_name) LIKE '%celular%' OR LOWER(prod_name) LIKE '%iphone%' ORDER BY   prod_unit_price DESC LIMIT 10;"

In [4]:
df = (client2.query(query).result().to_dataframe())


In [13]:
df = (client2.query(query.replace("GROUP_CONCAT", "STRING_AGG")).result().to_dataframe())

In [14]:
df

,prod_name,prod_unit_price


In [10]:
df_string = df.to_string()

In [11]:
df_string

'   int64_field_0      date        prod_id                                 prod_name                                           prod_name_long       prod_brand      category                     subcategory                   tags  prod_unit_price prod_units                                       prod_icon prod_source  source_type\n0          15852  20191101  2996131000003             queso semimaduro feta cheese                      queso semimaduro feta cheese x 210 g  centurion foods  supermercado                     charcutería  quesos especializados       10119047.0   unidades                 ../icons/SVG/74-food/cheese.svg       verde            1\n1          15845  20191101  2959274000002                   queso porvolone alpina                             queso porvolone alpina x 170g           alpina  supermercado                     charcutería  quesos especializados        8138823.0   unidades                 ../icons/SVG/74-food/cheese.svg       verde            1\n2          1